In [1]:
import pymp
from multiprocessing import Manager, cpu_count 
import sys
import os
import glob 
import gezi
from gezi import tqdm
import pandas as pd
import json
# import cudf
# import dask.dataframe as dd
import tensorflow as tf
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', None, '')
import melt

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#setting offilne
plotly.offline.init_notebook_mode(connected=True)

tensorflow_version: 1.14.0
torch_version: 1.3.1+cu100
Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
# 载入模型
model_hour = '2020010112'
model_root = '/home/gezi/tmp/rank/data/tuwen_hour_sgsapp_v2/models/15/'
FLAGS(sys.argv)
melt.init(tf.Graph())
sess = melt.get_session()
model_path = os.path.realpath(f'{model_root}/{model_hour}')
print(model_path)
predictor = melt.Predictor(sess=sess)
predictor.restore(model_path)
assert predictor.graph.get_collection('index_feed')

2020-01-03 21:22:54 0:00:00 os.environ['CUDA_VISIBLE_DEVICES'] -1
2020-01-03 21:22:54 0:00:00 os.environ['CUDA_VISIBLE_DEVICES'] -1
2020-01-03 21:22:54 0:00:00 Tf dataset and Tf model train in Graph mode, support Horovod, keras False
2020-01-03 21:22:54 0:00:00 seed 39321
2020-01-03 21:22:54 0:00:00 log_level: 20
2020-01-03 21:22:54 0:00:00 valid_interval_epochs: 0.0
2020-01-03 21:22:54 0:00:00 buffer_size:0
2020-01-03 21:22:54 0:00:00 batch_size: 32 eval_batch_size: 32 batch_size_per_gpu: 32 num_gpus: 1 horovod: False
2020-01-03 21:22:54 0:00:00 model_dir /tmp/melt log_dir /tmp/melt


/search/odin/publicData/CloudS/chenghuige/rank/data/tuwen_hour_sgsapp_v2/models/15/2020010112
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /search/odin/publicData/CloudS/chenghuige/rank/data/tuwen_hour_sgsapp_v2/models/15/2020010112/model.ckpt-2020010112-1.00-3487


In [3]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [4]:
marks = ['tuwen', 'video']
head = 'mid,docid,dur,pred,abid,ori_lr_score,lr_score,show_time,position,video_time,impression_time,interests,rea,pred_click,pred_dur'
mid = 'e5e9b96d57c21de54eecf437e2b4f4c3f1c7de66b950'
hour = '2020010113'

In [5]:
def deal(df):
  df.hour = df.hour.astype(str)
  df = df.groupby(['abtest', 'hour'], as_index=False).last()
  df = df.sort_values(by=['hour'])
  df = df.drop_duplicates()
  df.hour = pd.to_datetime(df.hour, format='%Y%m%d%H')
  return df

# 遍历infos路径获取scores信息
def gen_df(mark):
  file = f'/home/gezi/tmp/rank/data/{mark}_hour_sgsapp_v2/infos/15/{hour}/scores'  
  df = pd.read_csv(file, sep='\t', header=None)
  df.columns = head.split(',')[:len(df.columns)]
  return df

In [6]:
df_tuwen = gen_df('tuwen')
df_tuwen

,mid,docid,dur,pred,abid,ori_lr_score,lr_score,show_time,position,video_time,impression_time,interests,rea,pred_click,pred_dur
0,0000867359041845472,19c06g0f05iNQa,0,0.138567,5,0.078067,1.045850,1577856808,0,0,1577753365,0,931,0.069982,0.075241
1,0000867359041845472,open_doc_prod16437385,0,0.135167,5,0.472867,1.044700,1577856808,1,0,1577753365,0,931,0.047372,0.148663
2,0000867359041845472,open_doc_prod15837743,4,0.316967,5,0.354667,1.036600,1577856809,0,0,1577856809,0,931,0.250238,0.161059
3,0000867359041845472,open_doc_prod14870255,0,0.230067,5,0.293267,1.031010,1577856809,1,0,1577856809,0,931,0.173880,0.092663
4,0000867359041845472,open_doc_prod16447887,0,0.105067,5,0.093267,1.018980,1577856809,2,0,1577856809,0,931,0.037940,0.084534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1640576,ffff99001271794149,20191229A0360900,0,0.204367,7,0.212267,0.212267,1577856619,5,0,1577856373,4,909,0.239570,0.030377
1640577,ffff99001271794149,19c31m0l1183Uy,0,0.163967,7,0.199067,0.199067,1577856679,7,0,1577856373,8,905,0.152184,0.031224
1640578,ffff99001271794149,20200101A09RP700,73,0.172867,7,0.159367,0.159367,1577856693,10,0,1577856373,8,905,0.101830,0.086125
1640579,ffff99001271794149,19c31l0l3oQhNK,0,0.118167,7,0.146867,0.146867,1577856699,12,0,1577856373,8,905,0.038235,0.133213


In [7]:
df_video = gen_df('video')

In [8]:
df = df_tuwen
print(len(df))
print(len(df[df.impression_time==0]))
len(df[df.rea==0]) / len(df)

1640581
38517


0.03913247806722131

In [9]:
cold_starts = set([931,984,925,926])
base_ids = set([4,5,6])
df = df_tuwen
df = df[df.abid.isin(base_ids)]
df = df.fillna(0.)
df_ori = df
df_cold = df[df.rea.astype(int).isin(cold_starts)]
df_cold

,mid,docid,dur,pred,abid,ori_lr_score,lr_score,show_time,position,video_time,impression_time,interests,rea,pred_click,pred_dur
0,0000867359041845472,19c06g0f05iNQa,0,0.138567,5,0.078067,1.045850,1577856808,0,0,1577753365,0,931,0.069982,0.075241
1,0000867359041845472,open_doc_prod16437385,0,0.135167,5,0.472867,1.044700,1577856808,1,0,1577753365,0,931,0.047372,0.148663
2,0000867359041845472,open_doc_prod15837743,4,0.316967,5,0.354667,1.036600,1577856809,0,0,1577856809,0,931,0.250238,0.161059
3,0000867359041845472,open_doc_prod14870255,0,0.230067,5,0.293267,1.031010,1577856809,1,0,1577856809,0,931,0.173880,0.092663
4,0000867359041845472,open_doc_prod16447887,0,0.105067,5,0.093267,1.018980,1577856809,2,0,1577856809,0,931,0.037940,0.084534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1636028,ff67861184040884587,19c29l0l3s8v2G,0,0.081867,5,0.067367,0.067367,1577855169,2,0,1577855168,3,931,0.019617,0.116564
1636369,ff71869411024622416,open_doc_prod15352469,0,0.011867,4,0.030867,1.008260,1577855791,0,0,1576581100,3,931,0.000371,0.142080
1636370,ff71869411024622416,open_doc_prod13621487,0,0.004967,4,0.011367,1.007620,1577855792,1,0,1576581100,3,931,0.000072,0.118745
1636371,ff71869411024622416,19b17l0f1VcmnD,0,0.010567,4,0.016567,1.007370,1577855792,2,0,1576581100,3,931,0.000361,0.096078


In [10]:
# from projects.feed.rank.src.evaluate import evaluate_rank_ as eval
# from gezi import tqdm
# res = []
# result = eval(df, df_ori, df_cold, group=True, key='ori_lr_score')
# res += [result]

In [11]:
# from projects.feed.rank.src.evaluate import evaluate_rank_ as eval
# from gezi import tqdm
# result = eval(df, df_ori, df_cold, group=True, key='pred')
# res += [result]

In [12]:
# cols = ['gold/auc', 'auc', 'group/auc', 'click/time_auc', 'group/click/time_auc', 'cold/auc']
# pd.DataFrame.from_dict(res)[cols]

In [13]:
def predict(x):
  feed_dict = {
                predictor.graph.get_collection('index_feed')[0]: x['index'],
                predictor.graph.get_collection('value_feed')[0]: x['value'],
                predictor.graph.get_collection('field_feed')[0]: x['field'],
                predictor.graph.get_collection('uid_feed')[0]: x['uid'],
                predictor.graph.get_collection('did_feed')[0]: x['did'],
                predictor.graph.get_collection('doc_idx_feed')[0]: x['history'],
            } 
  return predictor.predict(['pred', 'pred_click', 'pred_dur'], feed_dict)

In [19]:
import multiprocessing
from projects.feed.rank.src.tfrecord_dataset import Dataset
from projects.feed.rank.src import util
from gezi import tqdm
tfrecord_root = '/home/gezi/tmp/rank/data/tuwen_hour_sgsapp_v2/tfrecords/'
tfrecord_dir = os.path.realpath(f'{tfrecord_root}/{hour}')
files = gezi.list_files(tfrecord_dir)
num_records_file = os.path.join(tfrecord_dir, "num_records.txt")
total = melt.get_num_records(files) if not os.path.exists(num_records_file) else gezi.read_int_from(num_records_file)
print('total', total, file=sys.stderr)
batch_size = 512
num_processes = 8
num_steps = -int(-total // (batch_size * num_processes))
print(num_steps)
manager = Manager()
res = manager.list()

with sess.graph.as_default():
  dataset = Dataset('valid')
  iters = [dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False, 
                            num_processes=num_processes, process_index=i) \
          for i in range(num_processes)]
  def deal(index):
    with tf.device('/cpu:%d' % index):
      iter = iters[index]
      op = iter.get_next()
      num_steps = -int(-total // batch_size)
      try:
        for _ in tqdm(range(num_steps), total=num_steps):
          x, y = sess.run(op)
          mids = np.asarray([m.decode() for m in x['mid']])
          docids = np.asarray([m.decode() for m in x['docid']])
          x['mid'] = mids
          x['docid'] = docids
      except Exception:
        pass

import threading
from gezi import Timer

timer = Timer('predict', True)
threads = []
for i in range(num_processes):
  threads.append(threading.Thread(target=deal,args=(i,)))

for t in threads:
#     t.setDaemon(True)
  t.start()
for t in threads:
  t.join()
timer.print_elapsed()

401


total 1640581
  5%|▍         | 145/3205 [04:21<1:37:37,  1.91s/it]




  4%|▍         | 129/3205 [04:21<1:53:07,  2.21s/it]


  4%|▍         | 129/3205 [04:21<1:53:16,  2.21s/it]



  4%|▍         | 129/3205 [04:21<1:53:17,  2.21s/it]

  5%|▌         | 163/3205 [07:59<1:47:25,  2.12s/it]

  5%|▌         | 163/3205 [07:59<1:47:28,  2.12s/it]


  5%|▌         | 163/3205 [08:31<1:47:39,  2.12s/it]

222
2


222


1
1
1
1
1
1
1


  5%|▍         | 146/3205 [05:43<1:37:33,  1.91s/it]


  4%|▍         | 130/3205 [04:23<1:48:48,  2.12s/it]

  4%|▍         | 130/3205 [04:23<1:48:56,  2.13s/it]



  4%|▍         | 130/3205 [04:23<1:48:57,  2.13s/it]




  5%|▌         | 164/3205 [08:33<1:51:30,  2.20s/it]


  5%|▌         | 164/3205 [08:33<1:51:28,  2.20s/it]

  5%|▌         | 164/3205 [08:33<1:51:27,  2.20s/it]

22222
2


2


1
1
1
1
1
1
1


...


  9%|▉         | 292/3205 [08:48<1:04:07,  1.32s/it]

 13%|█▎        | 401/3205 [13:17<56:37,  1.21s/it]  


 13%|█▎        | 401/3205 [14:37<56:54,  1.22s/it]  

 13%|█▎        | 401/3205 [16:54<56:59,  1.22s/it]  

 13%|█▎        | 401/3205 [13:17<57:07,  1.22s/it]  


 13%|█▎        | 401/3205 [13:17<56:59,  1.22s/it]  













2
22
222


2

1
1
1
1
1
1
1









  9%|▉         | 293/3205 [08:49<55:50,  1.15s/it]  

  9%|▉         | 294/3205 [08:50<48:51,  1.01s/it]

  9%|▉         | 295/3205 [08:50<43:49,  1.11it/s]

  9%|▉         | 296/3205 [08:51<41:00,  1.18it/s]

  9%|▉         | 296/3205 [08:51<41:07,  1.18it/s]

  9%|▉         | 298/3205 [08:53<38:24,  1.26it/s]

  9%|▉         | 299/3205 [08:53<37:42,  1.28it/s]

  9%|▉         | 300/3205 [08:54<36:57,  1.31it/s]

  9%|▉         | 301/3205 [08:55<36:54,  1.31it/s]

  9%|▉         | 302/3205 [08:56<36:43,  1.32it/s]

  9%|▉         | 303/3205 [08:56<36:44,  1.32it/s]

  9%|▉         | 304/3205 [08:57<35:59,  1.34it/s]

 10%|▉         | 305/3205 [08:58<36:31,  1.32it/s]

 10%|▉         | 306/3205 [08:59<36:16,  1.33it/s]

 10%|▉         | 307/3205 [08:59<36:42,  1.32it/s]

 10%|▉         | 308/3205 [09:00<36:42,  1.32it/s]

 10%|▉         | 309/3205 [09:01<36:12,  1.33it/s]

 10%|▉         | 310/3205 [09:02<35:38,  1.35it/s]

 10%|▉         | 311/3205 [09:02<35:49,  1.35it/s]

 10

 11%|█         | 342/3205 [09:25<34:35,  1.38it/s]

 11%|█         | 343/3205 [09:26<34:49,  1.37it/s]

 11%|█         | 344/3205 [09:26<34:59,  1.36it/s]

 11%|█         | 345/3205 [09:27<35:18,  1.35it/s]

 11%|█         | 346/3205 [09:28<34:24,  1.38it/s]

 11%|█         | 347/3205 [09:29<33:56,  1.40it/s]

 11%|█         | 348/3205 [09:29<33:30,  1.42it/s]

 11%|█         | 349/3205 [09:30<33:21,  1.43it/s]

 11%|█         | 350/3205 [09:31<33:15,  1.43it/s]

 11%|█         | 351/3205 [09:31<33:10,  1.43it/s]

 11%|█         | 352/3205 [09:32<33:00,  1.44it/s]

 11%|█         | 353/3205 [09:33<34:33,  1.38it/s]

 11%|█         | 354/3205 [09:34<35:22,  1.34it/s]

 11%|█         | 355/3205 [09:34<34:47,  1.37it/s]

 11%|█         | 356/3205 [09:35<35:28,  1.34it/s]

 11%|█         | 357/3205 [09:36<34:46,  1.37it/s]

 11%|█         | 358/3205 [09:37<34:52,  1.36it/s]

 11%|█         | 359/3205 [09:37<34:35,  1.37it/s]

 11%|█         | 360/3205 [09:38<33:54,  1.40it/s]

 11%|█▏     

 12%|█▏        | 391/3205 [11:01<3:13:27,  4.12s/it]

 12%|█▏        | 392/3205 [11:02<2:27:12,  3.14s/it]

 12%|█▏        | 393/3205 [11:04<2:06:17,  2.69s/it]

 12%|█▏        | 394/3205 [11:08<2:27:12,  3.14s/it]

 12%|█▏        | 395/3205 [11:10<2:11:52,  2.82s/it]

 12%|█▏        | 395/3205 [11:10<2:12:01,  2.82s/it]

 12%|█▏        | 396/3205 [11:17<3:06:12,  3.98s/it]

 12%|█▏        | 398/3205 [11:28<3:44:40,  4.80s/it]

 12%|█▏        | 399/3205 [11:29<2:47:29,  3.58s/it]

 12%|█▏        | 400/3205 [11:30<2:07:22,  2.72s/it]

 13%|█▎        | 401/3205 [11:30<1:34:16,  2.02s/it]

 13%|█▎        | 401/3205 [11:30<1:34:33,  2.02s/it]



In [15]:
import copy
l2 = copy.deepcopy(l)
for i in tqdm(range(len(l2))):
  filter_flag = l2[i]['mid'] == mid
#   filter_flag *= l2[i]['docid'] == '20101d0x0TZMcx'
  for key in l2[i]:
    try:
      l2[i][key] = l2[i][key][filter_flag]
    except Exception:
      pass
l2 = [x for x in l2 if len(x['mid'])]

  0%|          | 0/3205 [00:00<?, ?it/s]

  0%|          | 0/3205 [00:00<?, ?it/s]


  0%|          | 0/3205 [00:00<?, ?it/s]



  0%|          | 0/3205 [00:00<?, ?it/s]




  0%|          | 0/3205 [00:00<?, ?it/s]





  0%|          | 0/3205 [00:00<?, ?it/s]

NameError: name 'l' is not defined

  0%|          | 2/3205 [00:01<50:12,  1.06it/s]



  0%|          | 1/3205 [00:02<1:58:27,  2.22s/it]



  0%|          | 8/3205 [00:04<40:26,  1.32it/s]  



  0%|          | 9/3205 [00:05<37:47,  1.41it/s]t]



  0%|          | 6/3205 [00:05<1:03:51,  1.20s/it]



  0%|          | 11/3205 [00:06<33:11,  1.60it/s] 



  0%|          | 12/3205 [00:06<29:45,  1.79it/s][A



  0%|          | 10/3205 [00:06<35:53,  1.48it/s]




  0%|          | 1/3205 [00:07<6:15:17,  7.03s/it]


  0%|          | 1/3205 [00:07<6:14:35,  7.01s/it]

  0%|          | 1/3205 [00:07<6:14:51,  7.02s/it]





  0%|          | 13/3205 [00:07<29:12,  1.82it/s]]



  0%|          | 14/3205 [00:07<28:43,  1.85it/s]



  0%|          | 15/3205 [00:08<28:41,  1.85it/s]



  0%|          | 16/3205 [00:08<28:36,  1.86it/s]



  1%|          | 17/3205 [00:09<28:27,  1.87it/s]



  1%|          | 18/3205 [00:09<27:46,  1.91it/s]



  1%|          | 18/3205 [00:09<27:23,  1.94it/s]



  1%|          | 20/3205 [00:10<27:3

In [ ]:
len(l)

In [ ]:
predict(l2[0])

In [ ]:
x = copy.deepcopy(l2[0])
predict(x)

In [ ]:
def mask(x, fields):
  if isinstance(fields, str):
    fields = set(map(int, fields.split(',')))
  else:
    fields = set(fields)
  for i in range(len(x['field'])):
    flag = np.asarray([float(x !=0 and x not in fields) for x in x['field'][i]])
    x['value'][i] *= flag

In [ ]:
field_names = open('/home/gezi/mine/pikachu/projects/feed/rank/src/conf/tuwen/fields.txt').readlines()
field_names = [f'{i+1}:{x.strip()}' for i, x in enumerate(field_names)]
field_names.insert(0, '0:None')
# field_names

In [ ]:
res = []
for x in l2:
  pred = predict(x)
  res.append([x['docid'][0], x['impression_time'][0], x['position'][0], pred[0][0][0]])
df = pd.DataFrame.from_dict(res)
df.columns=['docid', 'impression_time', 'position', 'pred']
df.sort_values('pred', ascending=False)

In [ ]:
res = []
l2 = copy.deepcopy(l)
for x in l2:
  mask(x, [47,53,89,84,55])
  pred = predict(x)
  res.append([x['docid'][0], x['impression_time'][0], x['position'][0], pred[0][0][0]])
df = pd.DataFrame.from_dict(res)
df.columns=['docid', 'impression_time', 'position', 'pred']
df.sort_values('pred', ascending=False)

In [ ]:
res = []
for i in range(91):
  x = copy.deepcopy(l2[1])
  mask(x, [i])
  res.append([field_names[i], predict(x)[0][0][0]])
df = pd.DataFrame.from_dict(res)
df.columns=['field', 'pred']
df.sort_values('pred')

In [ ]:
x = copy.deepcopy(l2[0])
mask(x, [47,53,89,84,55])
predict(x)